# Prompt Engineering for Agentic AI

This notebook demonstrates prompt engineering techniques for working with agentic AI models. We'll explore different strategies to optimize how we communicate with AI models to achieve desired outcomes.

## Table of Contents
1. [Introduction to Prompt Engineering](#introduction)
2. [Environment Setup](#setup)
3. [Basic Prompt Structures](#basic-structures)
4. [Prompt Refinement Techniques](#refinement)
5. [Chain of Thought Prompting](#chain-of-thought)
6. [Advanced Patterns](#advanced-patterns)
7. [Summary and Best Practices](#summary)

## 1. Introduction to Prompt Engineering <a id="introduction"></a>

### What is Prompt Engineering?

Prompt engineering is the practice of designing and optimizing inputs (prompts) to AI language models to elicit desired responses. It's crucial for:

- **Accuracy**: Getting precise and relevant answers
- **Consistency**: Ensuring reliable outputs across similar queries
- **Control**: Directing the model's behavior and output format
- **Efficiency**: Reducing the need for multiple iterations

### Why It Matters for Agentic AI

In agentic AI systems, prompts serve as the primary interface for:
- Defining agent roles and behaviors
- Controlling task execution
- Managing multi-turn conversations
- Coordinating multiple agents

Well-crafted prompts can significantly improve agent performance, reliability, and user experience.

## 2. Environment Setup <a id="setup"></a>

First, let's set up our connection to Azure AI Foundry. Make sure you have:
1. Copied `.env.copy` to `.env`
2. Updated the values with your Azure AI project details

In [5]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Initialize the AI Project client
project_client = AIProjectClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential()
)

# Get the OpenAI client
chat = project_client.get_openai_client()
model = os.environ["MODEL"]

print(f"Connected to Azure AI Foundry")
print(f"Using model: {model}")

Connected to Azure AI Foundry
Using model: gpt-4.1-mini


In [6]:
# Helper function to display responses
def get_completion(messages, temperature=0.7, max_tokens=500):
    """Helper function to get completions from the model"""
    response = chat.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

def display_comparison(prompt1, prompt2, label1="Prompt 1", label2="Prompt 2"):
    """Display side-by-side comparison of two prompts and their responses"""
    print("=" * 80)
    print(f"\n{label1}:")
    print("-" * 80)
    print(f"Prompt: {prompt1[0]['content'] if isinstance(prompt1[0], dict) else prompt1}")
    response1 = get_completion(prompt1 if isinstance(prompt1[0], dict) else [{"role": "user", "content": prompt1}])
    print(f"\nResponse:\n{response1}")
    
    print("\n" + "=" * 80)
    print(f"\n{label2}:")
    print("-" * 80)
    print(f"Prompt: {prompt2[0]['content'] if isinstance(prompt2[0], dict) else prompt2}")
    response2 = get_completion(prompt2 if isinstance(prompt2[0], dict) else [{"role": "user", "content": prompt2}])
    print(f"\nResponse:\n{response2}")
    print("\n" + "=" * 80)

print("Helper functions loaded successfully!")

Helper functions loaded successfully!


## 3. Basic Prompt Structures <a id="basic-structures"></a>

Let's explore how different prompt structures affect model responses.

### 3.1 Vague vs. Specific Prompts

In [7]:
# Vague prompt
vague_prompt = "Tell me about Python."

# Specific prompt
specific_prompt = """Explain Python's list comprehension feature. 
Include:
1. A brief definition
2. Basic syntax
3. One practical example
Keep your response under 150 words."""

display_comparison(vague_prompt, specific_prompt, "Vague Prompt", "Specific Prompt")


Vague Prompt:
--------------------------------------------------------------------------------
Prompt: Tell me about Python.

Response:
Python is a high-level, interpreted programming language known for its readability, simplicity, and versatility. It was created by Guido van Rossum and first released in 1991. Python's design philosophy emphasizes code readability and a syntax that allows programmers to express concepts in fewer lines of code compared to languages like C++ or Java.

Key features of Python include:

1. **Easy to Learn and Use**: Python has a simple syntax similar to English, which makes it accessible to beginners.
2. **Interpreted Language**: Python code is executed line-by-line, which makes debugging easier.
3. **Dynamic Typing**: Variable types are determined at runtime, allowing for flexible coding.
4. **Extensive Standard Library**: Python comes with a large standard library that supports many common programming tasks such as file I/O, system calls, and internet pr

**Key Takeaway**: Specific prompts with clear instructions and constraints produce more focused and useful responses.

### 3.2 Role-Based Prompting

Assigning a role to the AI can significantly influence its response style and content.

In [ ]:
# No role assignment
no_role = [
    {"role": "user", "content": "How should I structure my code for a web application?"}
]

# With role assignment
with_role = [
    {"role": "system", "content": "You are a senior software architect with 15 years of experience in scalable web application design. Provide practical, industry-standard advice."},
    {"role": "user", "content": "How should I structure my code for a web application?"}
]

print("No Role Assignment:")
print("-" * 80)
response1 = get_completion(no_role)
print(response1)

print("\n" + "=" * 80)
print("\nWith Role Assignment (Senior Software Architect):")
print("-" * 80)
response2 = get_completion(with_role)
print(response2)
print("\n" + "=" * 80)

**Key Takeaway**: Role-based prompting helps set context, expertise level, and response style. The system message defines the AI's persona and behavior.

### 3.3 Output Format Control

You can control the structure and format of responses.

In [ ]:
# Unstructured request
unstructured = "List the benefits of using microservices architecture."

# Structured request
structured = """List the benefits of using microservices architecture.
Format your response as:
- Exactly 5 benefits
- Each benefit as a heading followed by a 1-sentence explanation
- Use numbered list format (1., 2., etc.)
"""

display_comparison(unstructured, structured, "Unstructured Format", "Structured Format")

**Key Takeaway**: Explicitly specifying output format ensures consistency and makes responses easier to parse and use in applications.

## 4. Prompt Refinement Techniques <a id="refinement"></a>

Let's explore techniques to refine prompts for better results.

### 4.1 Adding Context and Examples (Few-Shot Learning)

In [ ]:
# Zero-shot (no examples)
zero_shot = [
    {"role": "user", "content": "Classify the sentiment: 'This movie was absolutely terrible and I want my money back.'"}
]

# Few-shot (with examples)
few_shot = [
    {"role": "system", "content": "You are a sentiment classifier. Classify text as POSITIVE, NEGATIVE, or NEUTRAL."},
    {"role": "user", "content": "Classify: 'I love this product!'"},
    {"role": "assistant", "content": "POSITIVE"},
    {"role": "user", "content": "Classify: 'The weather is okay.'"},
    {"role": "assistant", "content": "NEUTRAL"},
    {"role": "user", "content": "Classify: 'This movie was absolutely terrible and I want my money back.'"}
]

print("Zero-Shot Approach:")
print("-" * 80)
response1 = get_completion(zero_shot)
print(response1)

print("\n" + "=" * 80)
print("\nFew-Shot Approach (with examples):")
print("-" * 80)
response2 = get_completion(few_shot)
print(response2)
print("\n" + "=" * 80)

**Key Takeaway**: Providing examples (few-shot learning) helps the model understand the expected output format and style, leading to more consistent results.

### 4.2 Using Delimiters and Structure

Clear delimiters help the model distinguish between instructions and content.

In [ ]:
# Without delimiters
without_delimiters = """Summarize the following text in one sentence: 
The quick brown fox jumps over the lazy dog. This sentence contains every letter of the alphabet. 
It's commonly used as a typing test."""

# With delimiters
with_delimiters = """Summarize the text delimited by triple backticks in one sentence.

Text: ```
The quick brown fox jumps over the lazy dog. This sentence contains every letter of the alphabet. 
It's commonly used as a typing test.
```

Summary:"""

display_comparison(without_delimiters, with_delimiters, "Without Delimiters", "With Delimiters")

**Key Takeaway**: Delimiters (```, ---, ###, etc.) help separate instructions from content, reducing ambiguity and improving accuracy.

### 4.3 Temperature Control for Creativity vs. Consistency

In [ ]:
prompt = "Write a creative tagline for a coffee shop that specializes in artisanal brews."

print("Low Temperature (0.2) - More Focused and Deterministic:")
print("-" * 80)
response_low = get_completion([{"role": "user", "content": prompt}], temperature=0.2)
print(response_low)

print("\n" + "=" * 80)
print("\nMedium Temperature (0.7) - Balanced:")
print("-" * 80)
response_med = get_completion([{"role": "user", "content": prompt}], temperature=0.7)
print(response_med)

print("\n" + "=" * 80)
print("\nHigh Temperature (1.2) - More Creative and Random:")
print("-" * 80)
response_high = get_completion([{"role": "user", "content": prompt}], temperature=1.2)
print(response_high)
print("\n" + "=" * 80)

**Key Takeaway**: 
- **Low temperature (0-0.3)**: Use for factual, consistent, and deterministic outputs
- **Medium temperature (0.5-0.8)**: Balanced approach for most use cases
- **High temperature (0.9-1.5)**: Use for creative, diverse outputs

## 5. Chain of Thought Prompting <a id="chain-of-thought"></a>

Chain of thought (CoT) prompting encourages the model to break down complex problems into steps, improving reasoning accuracy.

### 5.1 Basic Chain of Thought

In [ ]:
# Without chain of thought
direct_prompt = """A store sells apples for $2 each and oranges for $3 each. 
If John buys 5 apples and 3 oranges, and pays with a $50 bill, how much change does he get?"""

# With chain of thought
cot_prompt = """A store sells apples for $2 each and oranges for $3 each. 
If John buys 5 apples and 3 oranges, and pays with a $50 bill, how much change does he get?

Let's solve this step by step:"""

display_comparison(direct_prompt, cot_prompt, "Direct Answer", "Chain of Thought")

**Key Takeaway**: Adding "Let's solve this step by step" or similar phrases encourages the model to show its reasoning process, which often leads to more accurate answers.

### 5.2 Few-Shot Chain of Thought

Combining few-shot learning with chain of thought for complex reasoning tasks.

In [ ]:
# Few-shot CoT example
few_shot_cot = [
    {"role": "system", "content": "You are a helpful assistant that solves problems step by step."},
    {"role": "user", "content": "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"},
    {"role": "assistant", "content": """Let me solve this step by step:
1. Roger starts with 5 tennis balls
2. He buys 2 cans of tennis balls
3. Each can has 3 balls, so 2 cans have: 2 × 3 = 6 balls
4. Total tennis balls: 5 (initial) + 6 (new) = 11 balls

Answer: Roger has 11 tennis balls."""},
    {"role": "user", "content": """A cafeteria has 23 apples. If they used 20 apples to make lunch 
and bought 6 more, how many apples do they have now?"""}
]

print("Few-Shot Chain of Thought:")
print("-" * 80)
response = get_completion(few_shot_cot, max_tokens=300)
print(response)
print("\n" + "=" * 80)

**Key Takeaway**: Showing examples of step-by-step reasoning teaches the model to apply the same structured approach to new problems.

### 5.3 Self-Consistency with Chain of Thought

Generate multiple reasoning paths and choose the most consistent answer.

In [ ]:
problem = """A farmer has 17 sheep. All but 9 die. How many sheep does the farmer have left?
Think through this carefully step by step."""

print("Generating Multiple Reasoning Paths:")
print("=" * 80)

for i in range(3):
    print(f"\nAttempt {i+1}:")
    print("-" * 80)
    response = get_completion([{"role": "user", "content": problem}], temperature=0.8)
    print(response)

print("\n" + "=" * 80)
print("\nNote: By generating multiple responses, we can identify the most consistent answer.")
print("This technique is particularly useful for problems with tricky wording.")

**Key Takeaway**: Self-consistency involves generating multiple reasoning paths with slightly higher temperature and selecting the most common answer. This improves reliability for complex reasoning tasks.

## 6. Advanced Patterns <a id="advanced-patterns"></a>

### 6.1 Instruction Following with Constraints

In [ ]:
constrained_prompt = """Write a product description for a smart water bottle.

Constraints:
- Exactly 3 sentences
- First sentence: main benefit
- Second sentence: key features
- Third sentence: call to action
- Use enthusiastic but professional tone
- Do not use exclamation marks
"""

print("Response with Multiple Constraints:")
print("-" * 80)
response = get_completion([{"role": "user", "content": constrained_prompt}])
print(response)
print("\n" + "=" * 80)

**Key Takeaway**: Multiple constraints help shape the exact output you need. Be specific and clear about each requirement.

### 6.2 Iterative Refinement

Using conversation history to refine outputs.

In [ ]:
# Initial request
conversation = [
    {"role": "user", "content": "Write a haiku about programming."}
]

print("Initial Response:")
print("-" * 80)
response1 = get_completion(conversation)
print(response1)

# Add to conversation and refine
conversation.append({"role": "assistant", "content": response1})
conversation.append({"role": "user", "content": "Make it more technical and mention debugging."})

print("\n" + "=" * 80)
print("\nRefined Response:")
print("-" * 80)
response2 = get_completion(conversation)
print(response2)

# Further refinement
conversation.append({"role": "assistant", "content": response2})
conversation.append({"role": "user", "content": "Add a metaphor about nature."})

print("\n" + "=" * 80)
print("\nFurther Refined Response:")
print("-" * 80)
response3 = get_completion(conversation)
print(response3)
print("\n" + "=" * 80)

**Key Takeaway**: Multi-turn conversations allow iterative refinement. Each turn builds on previous context to progressively improve the output.

### 6.3 Meta-Prompting: Asking the Model to Improve Prompts

In [ ]:
meta_prompt = """I want to ask an AI to help me write better documentation for my code.
My current prompt is: "Write documentation for my function."

Please suggest an improved version of this prompt that would get better results. 
Explain why your version is better."""

print("Meta-Prompting - Asking AI to Improve Prompts:")
print("-" * 80)
response = get_completion([{"role": "user", "content": meta_prompt}], max_tokens=400)
print(response)
print("\n" + "=" * 80)

**Key Takeaway**: The model can help you improve your own prompts! This technique is useful when you're not sure how to phrase a request for optimal results.

## 7. Summary and Best Practices <a id="summary"></a>

### Key Principles of Effective Prompt Engineering

1. **Be Specific**: Clearly state what you want, including format, length, and style
2. **Provide Context**: Use system messages to set roles and expectations
3. **Use Examples**: Few-shot learning dramatically improves consistency
4. **Structure Your Prompts**: Use delimiters and clear sections
5. **Control Temperature**: Adjust based on your need for creativity vs. consistency
6. **Encourage Reasoning**: Use chain of thought for complex tasks
7. **Iterate and Refine**: Use conversation history to progressively improve outputs
8. **Set Constraints**: Define boundaries for length, format, tone, etc.

### Best Practices for Agentic AI

- **Define Clear Roles**: Each agent should have a well-defined purpose and expertise
- **Use Consistent Formats**: Standardize input/output formats across agents
- **Plan for Edge Cases**: Include instructions for handling unexpected inputs
- **Monitor and Log**: Track prompt performance and iterate based on results
- **Test Systematically**: Validate prompts across diverse scenarios

### Next Steps

- Experiment with different prompt patterns for your use case
- Build a library of effective prompts for common tasks
- Measure and compare prompt performance quantitatively
- Explore advanced techniques like prompt chaining and agent orchestration

### Additional Resources

- [Azure AI Foundry Documentation](https://learn.microsoft.com/azure/ai-studio/)
- [OpenAI Best Practices](https://platform.openai.com/docs/guides/prompt-engineering)
- [Prompt Engineering Guide](https://www.promptingguide.ai/)

## Practice Exercise

Try creating your own prompts for the following scenarios:

1. **Code Review Agent**: Design a prompt for an agent that reviews code and provides constructive feedback
2. **Data Analyst Agent**: Create a prompt for analyzing sales data and providing insights
3. **Customer Support Agent**: Build a prompt for handling customer inquiries with empathy and efficiency

Use the techniques learned in this notebook to craft effective prompts!

In [ ]:
# Your practice code here
# Example template:

practice_prompt = [
    {"role": "system", "content": "Your system message here"},
    {"role": "user", "content": "Your user message here"}
]

# Uncomment to test:
# response = get_completion(practice_prompt)
# print(response)